<b style='font-size:2em'>Using the py-droplets package</b>

This tutorial introduce the basic functions of the `py-droplets` package

In [ ]:
import numpy as np

# import necessary packages
import pde

import droplets

## Individual droplets

In the simplest case, a PDE evolves a field in time. To reflect this in code, we first define an initial state and then use a simple PDE to evolve it in time.

In [ ]:
drop1 = droplets.SphericalDroplet(position=[3, 4], radius=2)
drop1.volume

In [ ]:
grid = pde.CartesianGrid([[0, 10], [0, 10]], 64)
field1 = drop1.get_phase_field(grid)
field1.plot(colorbar=True)

In [ ]:
drop2 = droplets.DiffuseDroplet(position=[6, 8], radius=2, interface_width=0.3)
drop2.volume

In [ ]:
field2 = drop2.get_phase_field(grid)
field2.plot(colorbar=True)

In [ ]:
(field1 + field2).plot();

In [ ]:
drop1.overlaps(drop2)

## Emulsions
Emulsions are collection of droplets

In [ ]:
emulsion = droplets.Emulsion([drop1, drop2])
emulsion.get_size_statistics()

In [ ]:
emulsion.get_pairwise_distances()

In [ ]:
emulsion.plot()

In [ ]:
grid = pde.CartesianGrid([[0, 10], [0, 10]], 64)
emulsion.grid = grid
emulsion.plot()

In [ ]:
emulsion.get_phasefield(grid).plot(title="An emulsion");

In [ ]:
data = [
    droplets.SphericalDroplet(
        position=np.random.uniform(0, 100, 2), radius=np.random.uniform(5, 10)
    )
    for _ in range(30)
]

emulsion = droplets.Emulsion(data)
emulsion.remove_overlapping()
emulsion.plot(title=f"{len(emulsion)} droplets");

In [ ]:
emulsion.get_phasefield(pde.UnitGrid([100, 100])).plot(title="An emulsion");

## Analyze simulations
The `droplets` package can also be used to locate droplets in images, e.g., stemming from numerical simulations.

In [ ]:
# run a numerical simulation
grid = pde.UnitGrid([64, 64], periodic=True)
field = pde.ScalarField.random_uniform(grid, -1, 0)
eq = pde.CahnHilliardPDE()

final = eq.solve(field, t_range=1e2, dt=0.01);

In [ ]:
final.plot(title="Final phase field");

Locate the droplets in the final frame

In [ ]:
emulsion = droplets.locate_droplets(final)
emulsion[0]

In [ ]:
emulsion.get_size_statistics()

In [ ]:
emulsion.plot(field=final);

Locate and refine the droplets by fitting

In [ ]:
emulsion = droplets.locate_droplets(final, refine=True)
emulsion[0]

In [ ]:
emulsion.interface_width

Track the droplets during the simulation

In [ ]:
tracker = droplets.DropletTracker(interrupts=50)
final = eq.solve(field, t_range=1e3, dt=0.01, tracker=["progress", tracker]);

In [ ]:
final.plot();

In [ ]:
emulsion_last = tracker.data[-1]
emulsion_last.get_size_statistics()

In [ ]:
emulsion_last.plot(field=final, linewidth=3)

In [ ]:
for t, emulsion in zip(tracker.data.times, tracker.data, strict=False):
    print(f"t={round(t)}: {len(emulsion)} droplets")

Analyze droplet trajectories by identifying the dynamics of each droplet

In [ ]:
track_list = droplets.DropletTrackList.from_emulsion_time_course(tracker.data)

In [ ]:
track_list.plot()

In [ ]:
track_list.plot_positions()